# Load environment

In [ ]:
import pandas as pd
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import yaml
import numpy as np
from collections import defaultdict
from scipy.stats import variation, entropy

In [ ]:
with open("../data/resources/rcParams.yaml") as f:
    rcParamsDict = yaml.full_load(f)
    for k in rcParamsDict["rcParams"]:
        print("{} {}".format(k,rcParamsDict["rcParams"][k]))
        plt.rcParams[k] = rcParamsDict["rcParams"][k]
    for k1 in set(list(rcParamsDict)).difference(set(["rcParams"])):
        print("{} {}".format(k1,rcParamsDict[k1]))

In [ ]:
first_tranche = glob.glob("../../CensusSeq_Feb/output/census/**.txt")
second_tranche = glob.glob("../../CensusSeq_Mar/output/census/**.txt")

all_files = first_tranche + second_tranche

In [ ]:
color_palette = {
 'CTL01A': '#DBB807',
    'CTL08A': '#0FB248',
    'CTL04E': '#FF0054',
    'CTL02A': '#7B00FF',
'H9': '#72190E',
 'H1': '#994F88',
 'CTL05A': '#1965B0',
 'CTL07C': '#437DBF',
 'CTL06F': '#CAE0AB',
 'CTL09A': '#FFFF00',
 'KTD8_2': '#E65518',
 'UCSFi001-A': '#7BAFDE'}

# Load and format data

In [ ]:
all_results = {}
for file in all_files:
    name = file.strip('.txt').split('/')[-1].split('.')[0]
    all_results[name] = pd.read_csv(file, skiprows = 2, sep = '\t')

In [ ]:
all_results_df = pd.concat(all_results.values(), keys = all_results.keys()).reset_index()
donor_map_names = {i:j for i, j in zip(all_results_df['DONOR'], all_results_df['DONOR'])}
donor_map_names['CHD2WT'] = 'UCSFi001-A'
donor_map_names['CHD8WT'] = 'H9'
all_results_df['DONOR'] = all_results_df['DONOR'].map(donor_map_names)

all_results_df.head()

In [ ]:
all_results_df.shape 

In [ ]:
metadata = pd.read_excel('../../data/csv/CensusSeq_metadata_new.xlsx')
metadata

In [ ]:
metadata.index = metadata['Sample name']

In [ ]:
metadata['Mix'].unique()

In [ ]:
len(metadata['Sample name'].unique())

In [ ]:
len(all_results_df['level_0'].unique())

In [ ]:
all_results_df = all_results_df.drop('level_1', axis  = 1)
all_results_df.columns = ['Sample name', 'DONOR', 'REPRESENTATION']
all_results_df

In [ ]:
all_results_df.index = all_results_df['Sample name']
all_results_df['MIX ID'] = all_results_df['Sample name'].map({i: j for i, j in zip(metadata['Sample name'], metadata['Mix'])})
all_results_df['Timepoint'] = all_results_df['Sample name'].map({i: j for i, j in zip(metadata['Sample name'], metadata['timepoint'])})
all_results_df = all_results_df[all_results_df['MIX ID'] != 7]
#all_results_df['MIX ID'] = all_results_df['MIX ID'].map({1:1, 2:2, 3:3,4:4,5:5,6:6,8:7})
all_results_df.to_csv('../../data/csv/CensusSeq_data.csv')

In [ ]:
all_results_df.head()

In [ ]:
all_results_df.shape

# Plot contributions from each samples

In [ ]:
#color_palette

In [ ]:
order = ['day -2', 'day 5', 'day 12', 'day 25', 'day 50']
tp = all_results_df['MIX ID'].unique().tolist()
tp.sort()

fig, ax = plt.subplots(2, 4, figsize = (20, 20), gridspec_kw={'wspace': 0.4, 'hspace': 0.4})
ax = ax.flatten().T

for mix, ax in zip(tp, ax):
    
    sub = all_results_df[all_results_df['MIX ID'] == mix]
    #print(sub)
    sub_df_pivoted = pd.pivot(sub, index = 'Sample name', columns='DONOR', values='REPRESENTATION')
    
    sub_df_pivoted.index = sub_df_pivoted.index.map({i: j for i, j in zip(sub['Sample name'], sub['Timepoint'])})
    

    sub_df_pivoted.loc[[i for i in order if i in sub_df_pivoted.index]].plot(kind = 'bar', stacked = True, color = color_palette, ax = ax)
    ax.legend(bbox_to_anchor = (1,1))
    ax.set_title(f'Mix ID: {mix}')
    ax.set_xlabel('Time point')
    ax.set_ylabel('Proportion of identities')
#plt.savefig('censusSeq_results.png')
plt.tight_layout()
plt.show()

## Lineplots of contributions trends

In [ ]:
all_results_df['Timepoint_int'] = all_results_df.Timepoint.apply(lambda x: int(x.strip('day ')))
all_results_df['Timepoint_int'].sort_values
mix_ordered = all_results_df['MIX ID'].unique().tolist()
mix_ordered.sort()

fig, ax = plt.subplots(4, 2, figsize = (20, 25))
ax = ax.flatten().T

ax[-1].set_axis_off()

for mix, ax in zip(mix_ordered, ax):
    
    sub = all_results_df[all_results_df['MIX ID'] == mix].reset_index(drop = True)

    sns.lineplot(data = sub, x = 'Timepoint_int', y = 'REPRESENTATION', hue = 'DONOR', marker = 'o', palette=color_palette, ax = ax)
            
    ax.set_title(f'Mix ID: {mix}', fontsize = 25)
    ax.set_ylabel('Fraction represented', fontsize = 20)
    ax.set_xlabel('Timepoint', fontsize = 20)
    #start, end = (-3, 52)
    ax.xaxis.set_ticks([-2, 5, 12, 25, 50])

    ax.tick_params(axis='both', labelsize=15)
    
    
plt.tight_layout()
plt.savefig('./figures/Mix_CensusSeq.svg', bbox_inches = 'tight')

In [ ]:
all_results_df = all_results_df.drop('Sample name', axis = 1).reset_index()
max_rep = all_results_df[all_results_df.Timepoint == 'day 5'].groupby('Sample name').max('REPRESENTATION')['REPRESENTATION']
all_results_df[all_results_df.Timepoint == 'day 5'][all_results_df[all_results_df.Timepoint == 'day 5'].REPRESENTATION.isin(max_rep)]

In [ ]:
all_results_df.Timepoint.unique()

In [ ]:
fig, ax = plt.subplots(figsize = (20,5))
sns.boxplot(data = all_results_df, x = 'DONOR', y = 'REPRESENTATION', hue = 'Timepoint', hue_order=['day -2', 'day 5', 'day 12', 'day 25', 'day 50'])
ax.legend(bbox_to_anchor = (1,1))

# Compute entropy of representations
[Shannon's entropy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.entropy.html) is computed for each timepoint and each sample. It quantifies the expected uncertainty inherent in the possible outcomes of a discrete random variable, therefore the higher its value and and the higher the balance in representation. In fact, we observe a higher entropy for -2 time point.

In [ ]:
entropy_df = all_results_df.groupby(['Timepoint_int', 'Sample name'])['REPRESENTATION'].apply(entropy).reset_index()
entropy_df.head()

In [ ]:
fig, ax = plt.subplots(figsize = (5,8))
sns.barplot(data = entropy_df, x = 'Timepoint_int', y = 'REPRESENTATION', ax = ax, color = '#2a9d8f')
#sns.swarmplot(data = std, x = 'MIX ID', y = 'REPRESENTATION', hue = 'Timepoint_int', ax = ax, dodge = True)
ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)

ax.set_ylabel('Shannon entropy', fontsize = 20)
ax.set_xlabel('Day', fontsize = 20)
ax.tick_params(axis='both', which='major', labelsize=20)

#plt.legend(bbox_to_anchor = (1,1), title = 'Day')
plt.tight_layout()
plt.savefig('./figures/Entropy.svg', bbox_inches = 'tight')

# Compute coefficient of variation of representations for each donor
[Coefficient of variation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.variation.html#scipy.stats.variation) is computed for __each timepoint and each donor__ (so variation for each donor at a certain time point across different mixes). Time point -2 is excluded because no replicate i available for that.  
Additionally, for each donor the absolute difference in CV between one time point and the previous one is computed.

In [ ]:
std = all_results_df.groupby(['Timepoint_int', 'DONOR'])['REPRESENTATION'].apply(variation).reset_index()
std = std[std.Timepoint_int != -2]

std = std.sort_values(by=["DONOR", "Timepoint_int"])
std["Difference"] = std.groupby("DONOR")["REPRESENTATION"].diff()
#std

In [ ]:
fig, ax = plt.subplots(figsize = (20,10))
sns.barplot(data = std, x = 'DONOR', y = 'REPRESENTATION', hue = 'Timepoint_int', ax = ax, palette = sns.cubehelix_palette())
ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)
plt.legend(bbox_to_anchor = (1,1))
plt.savefig('./figures/CV_timepoint_donor.svg', bbox_inches = 'tight')

In [ ]:
fig, ax = plt.subplots(figsize = (20,10))
std['Difference'] = np.abs(std['Difference'])
sns.barplot(data = std, x = 'DONOR', y = 'Difference', hue = 'Timepoint_int', ax = ax, palette = sns.cubehelix_palette())
ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)
plt.legend(bbox_to_anchor = (1,1))

# Compute coefficient of variation the of representations for each donor in each mix
[Coefficient of variation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.variation.html#scipy.stats.variation) (CV) is computed for __each timepoint, each mix and each donor__ (so variation across different replicates). Time point -2 is excluded because no replicate i available for that.  
Additionally, for each donor the absolute difference in CV between one time point and the previous one is computed.

In [ ]:
std = all_results_df.groupby(['Timepoint_int', 'MIX ID', 'DONOR'])['REPRESENTATION'].apply(variation).reset_index()
std = std[std.Timepoint_int != -2]

std = std.sort_values(by=["MIX ID", 'DONOR', "Timepoint_int"])
std["Difference"] = std.groupby("MIX ID")["REPRESENTATION"].diff()
std

We plot here the distribution of the CV in each mix for each timepoint (each point of the distribution would be a donor)

In [ ]:
fig, ax = plt.subplots(figsize = (20,10))
sns.boxplot(data = std, x = 'MIX ID', y = 'REPRESENTATION', hue = 'Timepoint_int', ax = ax, palette = sns.cubehelix_palette())
#ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)
ax.set_ylabel('Coefficient of variation', fontsize = 20)
ax.set_xlabel('MIX', fontsize = 20)

ax.tick_params(axis='both', which='major', labelsize=20)
plt.tight_layout()
plt.legend(bbox_to_anchor = (1,1), title = 'Day')
plt.savefig('./figures/CV_timepoint_mix_donor.svg', bbox_inches = 'tight')

In [ ]:
std['Difference'] = np.abs(std['Difference'])

fig, ax = plt.subplots(figsize = (20,10))
sns.boxplot(data = std, x = 'MIX ID', y = 'Difference', hue = 'Timepoint_int', ax = ax, palette = sns.cubehelix_palette())
ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)
ax.set_ylabel('Absolute CV difference')
plt.legend(bbox_to_anchor = (1,1), title = 'Time range')

# Compute weighted rank and weighted normalized rank
## Day 5

In [ ]:
all_timepoints_norm_weighted = {}

In [ ]:
ranked_lists = {}
for m in all_results_df[all_results_df['Timepoint'] == 'day 5']['MIX ID'].unique():
    df = all_results_df[((all_results_df['Timepoint'] == 'day 5') & (all_results_df['MIX ID'] == m)  )].sort_values(by = 'REPRESENTATION', ascending = False)
    #print(df.groupby('DONOR').sum().sort_values(by = 'REPRESENTATION', ascending = False))
    result = df.groupby('DONOR').sum().sort_values(by = 'REPRESENTATION', ascending = False).reset_index()
    ranked_lists[f'Mix {m}'] = result.DONOR.tolist()

In [ ]:
ranked_lists

In [ ]:

data = list(ranked_lists.values())


### Weighted mean - not including percentages

In [ ]:
d = defaultdict(list)
d

for l in data:
    #print(len(l))
    for idx, value in enumerate(l):
        d[value].append( (idx + 1) * len(l)) # or (idx + 1) * len(l)? which one is better?

In [ ]:
mean_d = {}
for l in d:
    mean_d[l] = np.mean(d[l])

In [ ]:
mean_d_df = pd.DataFrame(mean_d.values(), mean_d.keys())
mean_d_df.columns = ['mean_weighted_rank']
#mean_d_df = mean_d_df.drop(1)
mean_d_df.sort_values(by = 'mean_weighted_rank', ascending = True)

In [ ]:
mean_d_df.sort_values(by = 'mean_weighted_rank').to_csv('../../data/csv/CensusSeq_weighted_rank_d5.csv')

### Weighted mean - including percentages

In [ ]:
all_results_df[(all_results_df['Timepoint'] == 'day 5')].groupby('DONOR').sum('REPRESENTATION').sort_values(by = 'REPRESENTATION')

In [ ]:
mean_d_df['mean_Representation'] = all_results_df[(all_results_df['Timepoint'] == 'day 5')].groupby('DONOR').mean('REPRESENTATION')['REPRESENTATION']
mean_d_df['combined_scores'] = mean_d_df['mean_weighted_rank'] * mean_d_df['mean_Representation']
mean_d_df.sort_values('combined_scores')

In [ ]:
mean_d_df.sort_values('combined_scores').to_csv('../../data/csv/CensusSeq_combined_weighted_rank_d5.csv')

In [ ]:
np.round(mean_d_df.sort_values('combined_scores'), 2)

In [ ]:
all_timepoints_norm_weighted['day 5'] = mean_d_df['combined_scores']

## Day 12

In [ ]:
ranked_lists = {}
for m in all_results_df[all_results_df['Timepoint'] == 'day 12']['MIX ID'].unique():
    df = all_results_df[((all_results_df['Timepoint'] == 'day 12') & (all_results_df['MIX ID'] == m)  )].sort_values(by = 'REPRESENTATION', ascending = False)
    result = df.groupby('DONOR').sum().sort_values(by = 'REPRESENTATION', ascending = False).reset_index()
    ranked_lists[f'Mix {m}'] = result.DONOR.tolist()

In [ ]:
data = list(ranked_lists.values())

### Weighted mean - not including percentages

In [ ]:
d = defaultdict(list)
d

for l in data:
    for idx, value in enumerate(l):
        d[value].append( (idx + 1) * len(l)) # or (idx + 1) * len(l)? which one is better?

In [ ]:
mean_d = {}
for l in d:
    mean_d[l] = np.mean(d[l])

In [ ]:
mean_d_df = pd.DataFrame(mean_d.values(), mean_d.keys())
mean_d_df.columns = ['mean_weighted_rank']
mean_d_df.sort_values(by = 'mean_weighted_rank', ascending = True)

### Weighted mean - including percentages

In [ ]:
all_results_df[(all_results_df['Timepoint'] == 'day 12')].groupby('DONOR').sum('REPRESENTATION').sort_values(by = 'REPRESENTATION')

In [ ]:
mean_d_df['mean_Representation'] = all_results_df[(all_results_df['Timepoint'] == 'day 12')].groupby('DONOR').mean('REPRESENTATION')['REPRESENTATION']
mean_d_df['combined_scores'] = mean_d_df['mean_weighted_rank'] * mean_d_df['mean_Representation']
mean_d_df.sort_values('combined_scores')

In [ ]:
mean_d_df.sort_values('combined_scores').to_csv('../../data/csv/CensusSeq_combined_weighted_rank_d12.csv')

In [ ]:
np.round(mean_d_df.sort_values('combined_scores'), 2)

In [ ]:
all_timepoints_norm_weighted['day 12'] = mean_d_df['combined_scores']

## Day 25

In [ ]:
ranked_lists = {}
for m in all_results_df[all_results_df['Timepoint'] == 'day 25']['MIX ID'].unique():
    df = all_results_df[((all_results_df['Timepoint'] == 'day 25') & (all_results_df['MIX ID'] == m)  )].sort_values(by = 'REPRESENTATION', ascending = False)
    result = df.groupby('DONOR').sum().sort_values(by = 'REPRESENTATION', ascending = False).reset_index()
    ranked_lists[f'Mix {m}'] = result.DONOR.tolist()

In [ ]:
data = list(ranked_lists.values())

### Weighted mean - not including percentages

In [ ]:
d = defaultdict(list)
d

for l in data:
    for idx, value in enumerate(l):
        d[value].append( (idx + 1) * len(l)) # or (idx + 1) * len(l)? which one is better?

In [ ]:
mean_d = {}
for l in d:
    mean_d[l] = np.mean(d[l])

In [ ]:
mean_d_df = pd.DataFrame(mean_d.values(), mean_d.keys())
mean_d_df.columns = ['mean_weighted_rank']
mean_d_df.sort_values(by = 'mean_weighted_rank', ascending = True)

### Weighted mean - including percentages

In [ ]:
all_results_df[(all_results_df['Timepoint'] == 'day 25')].groupby('DONOR').sum('REPRESENTATION').sort_values(by = 'REPRESENTATION')

In [ ]:
mean_d_df['mean_Representation'] = all_results_df[(all_results_df['Timepoint'] == 'day 25')].groupby('DONOR').mean('REPRESENTATION')['REPRESENTATION']
mean_d_df['combined_scores'] = mean_d_df['mean_weighted_rank'] * mean_d_df['mean_Representation']
mean_d_df.sort_values('combined_scores')

In [ ]:
np.round(mean_d_df.sort_values('combined_scores'), 2)

In [ ]:
mean_d_df.sort_values('combined_scores').to_csv('../../data/csv/CensusSeq_combined_weighted_rank_d25.csv')

In [ ]:
all_timepoints_norm_weighted['day 25'] = mean_d_df['combined_scores']

## Day 50

In [ ]:
ranked_lists = {}
for m in all_results_df[all_results_df['Timepoint'] == 'day 50']['MIX ID'].unique():
    df = all_results_df[((all_results_df['Timepoint'] == 'day 50') & (all_results_df['MIX ID'] == m)  )].sort_values(by = 'REPRESENTATION', ascending = False)
    result = df.groupby('DONOR').sum().sort_values(by = 'REPRESENTATION', ascending = False).reset_index()
    ranked_lists[f'Mix {m}'] = result.DONOR.tolist()

In [ ]:
data = list(ranked_lists.values())

### Weighted mean - not including percentages

In [ ]:
d = defaultdict(list)
d

for l in data:
    for idx, value in enumerate(l):
        d[value].append( (idx + 1) * len(l)) # or (idx + 1) * len(l)? which one is better?

In [ ]:
mean_d = {}
for l in d:
    mean_d[l] = np.mean(d[l])

In [ ]:
mean_d_df = pd.DataFrame(mean_d.values(), mean_d.keys())
mean_d_df.columns = ['mean_weighted_rank']
mean_d_df.sort_values(by = 'mean_weighted_rank', ascending = True)

### Weighted mean - including percentages

In [ ]:
all_results_df[(all_results_df['Timepoint'] == 'day 50')].groupby('DONOR').sum('REPRESENTATION').sort_values(by = 'REPRESENTATION')

In [ ]:
mean_d_df['mean_Representation'] = all_results_df[(all_results_df['Timepoint'] == 'day 50')].groupby('DONOR').mean('REPRESENTATION')['REPRESENTATION']
mean_d_df['combined_scores'] = mean_d_df['mean_weighted_rank'] * mean_d_df['mean_Representation']
mean_d_df.sort_values('combined_scores')

In [ ]:
mean_d_df.sort_values('combined_scores').to_csv('../../data/csv/CensusSeq_combined_weighted_rank_d50.csv')

In [ ]:
np.round(mean_d_df.sort_values('combined_scores'), 2)

In [ ]:
all_timepoints_norm_weighted['day 50'] = mean_d_df['combined_scores']

In [ ]:
all_results_df[((all_results_df['MIX ID'] == 6) & (all_results_df['Timepoint'] == 'day 50'))].groupby(['DONOR']).mean()